In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_reservation = pd.read_parquet(datadir / "reservation.parquet")
pd_holiday     = pd.read_parquet(datadir / "holiday.parquet")

# 12章 日時
## 12-1 日時型への変換
### Q: 文字列の日時を日時型に変換


#### Awesome

In [ ]:
(
    pd.DataFrame({"datetime_str": ["2023-04-01 10:11:30"]})
    .assign(
        # （1） tz-naiveなtimestampへ変換
        timestamp_naive=lambda df:
            pd.to_datetime(df.datetime_str, format="%Y-%m-%d %H:%M:%S"),
        # （2） 文字列の左側10文字を取り出してtz-naiveなtimestampへ変換
        date=lambda df:
            pd.to_datetime(df.datetime_str.str[:10], format="%Y-%m-%d"),
        # （3） tz-awareなtimestampへ変換 (JST)
        timestamp_jst=lambda df:
            pd.to_datetime(df.datetime_str, format="%Y-%m-%d %H:%M:%S")
            .dt.tz_localize("Asia/Tokyo"),
        # （4） tz-awareなtimestampへ変換 (PST)
        timestamp_pst=lambda df:
            pd.to_datetime(df.datetime_str, format="%Y-%m-%d %H:%M:%S")
            .dt.tz_localize("America/Los_Angeles"),
    )
)

## 12-2 日時型の列を用いた行の抽出
### Q: 指定期間に予約された予約履歴の抽出


#### Not Awesome

In [ ]:
(
    pd_reservation
    .loc[lambda df: df.reserved_at.between("2019-07-01", "2019-08-31")]
)

#### Awesome

In [ ]:
(
    pd_reservation
    .loc[lambda df: df.reserved_at.dt.normalize()
        .between("2019-07-01", "2019-08-31")]
)

## 12-3 日時要素の抽出
### Q: 予約日時の日時要素を抽出


#### Not Awesome

In [ ]:
(
    pd_reservation
    [["reserved_at"]]
    .assign(
        # （1） 年を抽出
        reserved_at_year=lambda df: df.reserved_at.dt.year,
        # （2） 月を抽出
        reserved_at_month=lambda df: df.reserved_at.dt.month,
        # （3） 日を抽出
        reserved_at_day=lambda df: df.reserved_at.dt.day,
        # （4） 曜日を抽出 （週の最初を月曜日とした0〜6の数字）
        reserved_at_dayofweek=lambda df: df.reserved_at.dt.dayofweek,
        # （5） 週を抽出 （ISO 8601形式の週番号）
        reserved_at_isoweek=lambda df: df.reserved_at.dt.isocalendar().week,
        # （6） 時を抽出
        reserved_at_hour=lambda df: df.reserved_at.dt.hour,
        # （7） 分を抽出
        reserved_at_minute=lambda df: df.reserved_at.dt.minute,
        # （8） 秒を抽出
        reserved_at_second=lambda df: df.reserved_at.dt.second,
        # （9） 年月を抽出
        reserved_at_ym=lambda df: df.reserved_at.dt.strftime("%Y-%m"),
    )
)

#### Awesome

In [ ]:
(
    pd_reservation
    [["reserved_at"]]
    .assign(
        # （1） 年を抽出
        reserved_at_year=lambda df: df.reserved_at.dt.year,
        # （2） 月を抽出
        reserved_at_month=lambda df: df.reserved_at.dt.month,
        # （3） 日を抽出
        reserved_at_day=lambda df: df.reserved_at.dt.day,
        # （4） 曜日を抽出 （週の最初を月曜日とした0〜6の数字）
        reserved_at_dayofweek=lambda df: df.reserved_at.dt.dayofweek,
        # （5） 週を抽出 （ISO 8601形式の週番号）
        reserved_at_isoweek=lambda df: df.reserved_at.dt.isocalendar().week,
        # （6） 時を抽出
        reserved_at_hour=lambda df: df.reserved_at.dt.hour,
        # （7） 分を抽出
        reserved_at_minute=lambda df: df.reserved_at.dt.minute,
        # （8） 秒を抽出
        reserved_at_second=lambda df: df.reserved_at.dt.second,
        # （9） 年月を抽出
        reserved_at_ym=lambda df: df.reserved_at.dt.year.astype(str)
            + "-" + df.reserved_at.dt.month.astype(str).str.zfill(2),
    )
)

## 12-4 日時の丸め処理
### Q: 予約日時の切り捨て


#### Awesome

In [ ]:
(
    pd_reservation
    [["reserved_at"]]
    .assign(
        # （1） 年に切り捨て
        reserved_at_year=lambda df: df.reserved_at.dt.to_period("Y")
            .dt.to_timestamp(),
        # （2） 四半期に切り捨て
        reserved_at_quarter=lambda df: df.reserved_at.dt.to_period("Q")
            .dt.to_timestamp(),
        # （3） 月に切り捨て
        reserved_at_month=lambda df: df.reserved_at.dt.to_period("M")
            .dt.to_timestamp(),
        # （4） 日に切り捨て
        reserved_at_day=lambda df: df.reserved_at.dt.floor("D"),
        # （5） 時に切り捨て
        reserved_at_hour=lambda df: df.reserved_at.dt.floor("h"),
        # （6） 分に切り捨て
        reserved_at_minute=lambda df: df.reserved_at.dt.floor("min"),
    )
)

## 12-5 日時の加減算
### Q: 予約日時に一定時間を加算


#### Awesome

In [ ]:
(
    pd_reservation
    [["reserved_at"]]
    .assign(
        # （1） 30分を加算
        reserved_at_add30min=lambda df: df.reserved_at + pd.DateOffset(minutes=30),
        # （2） 1時間を加算
        reserved_at_add1h=lambda df: df.reserved_at + pd.DateOffset(hours=1),
        # （3） 1日を加算
        reserved_at_add1d=lambda df: df.reserved_at + pd.DateOffset(days=1),
        # （4） 1ヶ月を加算
        reserved_at_add1m=lambda df: df.reserved_at
            + pd.DateOffset(months=1, normalize=True),
        # （5） 1年を加算
        reserved_at_add1y=lambda df: df.reserved_at
            + pd.DateOffset(years=1, normalize=True),
    )
)

### Q: 予約日時とチェックイン日の時間差を計算


#### Awesome

In [ ]:
import numpy as np

(
    pd_reservation
    [["reserved_at", "checkin_date"]]
    .assign(
        # （1） 年数差を計算
        diff_years=lambda df: (
                (df.checkin_date.dt.year*12 + df.checkin_date.dt.month)
                - (df.reserved_at.dt.year*12 + df.reserved_at.dt.month)
                - np.where(df.checkin_date.dt.day < df.reserved_at.dt.day, 1, 0)
            ) // 12,
        # （2） 月数差を計算
        diff_months=lambda df: 
            (df.checkin_date.dt.year*12 + df.checkin_date.dt.month)
            - (df.reserved_at.dt.year*12 + df.reserved_at.dt.month)
            - np.where(df.checkin_date.dt.day < df.reserved_at.dt.day, 1, 0),
        # （3） 日数差を計算
        diff_days=lambda df: (df.checkin_date - df.reserved_at).dt.days,
        # （4） 時間差を計算
        diff_hours=lambda df:
            (df.checkin_date - df.reserved_at).dt.total_seconds() / (60*60),
        # （5） 分差を計算
        diff_minutes=lambda df:
            (df.checkin_date - df.reserved_at).dt.total_seconds() / 60,
        # （6） 秒差を計算
        diff_seconds=lambda df: (df.checkin_date - df.reserved_at).dt.total_seconds()
    )
)

## 12-6 日時の数値化
### Q: チェックイン日の休日フラグの付与


#### Awesome

In [ ]:
import numpy as np

# （1） 日付マスタの作成
date_master = (
    # （1）-1 連続した日付の生成
    pd.DataFrame({"dt": pd.date_range("2014-01-01", "2019-12-31")})
    # （1）-2 祝日マスタの結合
    .merge(pd_holiday, left_on="dt", right_on="holiday_date", how="left")
    .assign(
        dt=lambda df: df.dt.dt.tz_localize("Asia/Tokyo"),
        # （1）-3 休日フラグの作成
        is_day_off=lambda df: np.where(df.holiday_name.notnull()
                                       | df.dt.dt.dayofweek.isin([5, 6]), 1, 0),
        # （1）-4 次の日の休日フラグを取得して休日前日フラグを作成
        is_day_before_day_off=lambda df: df.is_day_off.shift(-1)
    )
    [["dt", "is_day_off", "is_day_before_day_off"]]
)

(
    pd_reservation
    # （2） checkin_dateを用いて日付マスタを結合
    .merge(date_master, left_on="checkin_date", right_on="dt", how="left")
    .drop(columns="dt")
)